In [1]:
import pandas as pd
import numpy as np 
import matplotlib as mlp 
from matplotlib import pyplot as plt 
import geopandas as gp
import xlrd 
import os

os.chdir("../../core")
import GT_helper_functions as hf
import GT_load_data as data

base_dir = "../../"

%matplotlib inline

In [34]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

In [11]:
# Valores de Organismo:
# 401 - Banco Centroamericano de Integracion Economica
# 2102 - Donantes varios externos
# 425 - Fondo Mundial

In [3]:
def nextCell(row, offset = 0):
    lenr = len(row)
    if offset >= lenr:
        return None
    while (offset+1 < lenr) & (row[offset].ctype == xlrd.XL_CELL_EMPTY):
        offset += 1
    return offset

mapCodes = {
    2: "entidad",
    3: "ft_org_cor",
    5: "subprograma",
    6: "actividad_o",
    7: "renglon",
    9: "municipio"
}
codes = {
    "entidad": None,
    "ft_org_cor": None,
    "subprograma": None,
    "actividad_o": None,
    "renglon": None,
    "municipio": None
}
cols = [
    "entidad",
    "ft_org_cor",
    "subprograma",
    "actividad_o",
    "renglon",
    "municipio",
    "ASIGNADO", "MODIFICADO", "VIGENTE", "PRE_COMPROMISO", 
    "COMPROMETIDO", "DEVENGADO", "PAGADO", "SALDO_POR_COMPROMETER", 
    "SALDO_POR_DEVENGAR", "SALDO_POR_PAGAR", "PorcentajeEJEC"
    ]
len(cols)

17

In [4]:
def SICOIN_extractTable(xlsName):
    xlsfile = xlrd.open_workbook(xlsName)
    sheet = xlsfile.sheet_by_index(0)
    i = 19
    data = []
    continuousEmpty = 0
    while True:
        i += 1
        if i >= sheet.nrows:
            break
        row = sheet.row(i)
        j = 0
        j = nextCell(row, j)
        j0 = j
        if j <= 9:
            continuousEmpty = 0

            code = str(row[j].value).strip()
            if code == "TOTAL":
                continue

            j = nextCell(row, j+1)
            name = str(row[j].value).strip()
            codename = code + "-" + name
            if (j0 in [5,6]):
                if (len(code) == 5):
                    codes["subprograma"] = codename
                else:
                    codes["actividad_o"] = codename

            codes[mapCodes[j0]] = codename

            j = nextCell(row, j+1)
            if j is None:
                continue
            if j == 63: 
                continue

            rowData = list(codes.values())
            while True:
                value = str(row[j].value).strip()
                rowData.append(value)
                j = nextCell(row, j+1)
                if j is None:
                    break

            data.append(rowData)
        elif j == 63:
            # empty row
            continuousEmpty += 1
        if continuousEmpty > 5:
            break;
    df = pd.DataFrame(data = data, columns = cols)
    return df

In [8]:
sicoin_files = [
    { "file": "SICOIN - MSPAS TB FR-ORG-COR SubP ActivOb Renglon Muni - 2017.xls", "year": 2017},
    { "file": "SICOIN - MSPAS VIH FR-ORG-COR SubP ActivOb Renglon Muni - 2017.xls", "year": 2017},
    { "file": "SICOIN - MSPAS VectoresZoontca - FR-ORG-COR SubP ActivOb Renglon Muni - 2017.xls", "year": 2017},
    { "file": "SICOIN - MSPAS TB FR-ORG-COR SubP ActivOb Renglon Muni - 2016.xls", "year": 2016},
    { "file": "SICOIN - MSPAS VIH FR-ORG-COR SubP ActivOb Renglon Muni - 2016.xls", "year": 2016},
    { "file": "SICOIN - MSPAS VectoresZoontca - FR-ORG-COR SubP ActivOb Renglon Muni - 2016.xls", "year": 2016},
    { "file": "SICOIN - MSPAS TB FR-ORG-COR SubP ActivOb Renglon Muni - 2015.xls", "year": 2015},
    { "file": "SICOIN - MSPAS VIH FR-ORG-COR SubP ActivOb Renglon Muni - 2015.xls", "year": 2015},
    { "file": "SICOIN - MSPAS VectoresZoontca - FR-ORG-COR SubP ActivOb Renglon Muni - 2015.xls", "year": 2015} 
]

dataset = None
for filed in sicoin_files:
    temp = SICOIN_extractTable("../../Resource Tracking/SICOIN GT/Raw Data/" + filed["file"])
    temp["Year"] = filed["year"]
    if dataset is None:
        dataset = temp
    else:
        dataset = pd.concat([dataset, temp])

In [9]:
dataset.DEVENGADO = dataset.DEVENGADO.astype(float)

In [14]:
dataset = dataset.assign(finsource = lambda x: x.ft_org_cor.map(lambda y: ( ("GF" if ("425" in y) else "Others" if ("2102" in y) else \
                                                     "BCIE" if "401" in y else "Unknown - Others") if y.startswith("61") \
                                                      else "GHE" ) ))\
#.groupby(["ft_org_cor"]).DEVENGADO.sum()

In [15]:
dataset["programa"] = dataset.subprograma.map(lambda x: x[0:2])

In [56]:
(dataset.groupby(["programa", "finsource", "Year"]).DEVENGADO.sum().unstack(1) / 1000000).round(2)\
        .assign(gfp = lambda x: x.GF.divide(x.sum(1), 1).round(2)*100 ).fillna(0)

finsource         GF     GHE  Others   gfp
programa Year                             
16       2015  13.64   62.80    0.00  18.0
         2016   6.92   78.53    0.00   8.0
         2017   2.56   97.18    0.00   3.0
17       2015   0.00   18.31    0.81   0.0
         2016   0.70   14.12    0.00   5.0
         2017   2.69   12.99    0.00  17.0
18       2015  22.34  115.79    0.00  16.0
         2016   6.70  138.22    0.00   5.0
         2017   6.71  151.79    0.00   4.0

In [58]:
temp = (dataset[dataset.finsource == "GF"].groupby(["Year", "programa", "actividad_o"]).DEVENGADO.sum().unstack([0,1]) / 1000000)
(temp.divide(temp.sum(0), 1).round(2) * 100).fillna(0).sort_values(temp.columns[-1], ascending=False)

Year                                                2015         2016         \
programa                                              16     18    16     17   
actividad_o                                                                    
18 00 000 001 000-PERSONA BENEFICIADA CON ACCIO...   0.0  100.0   0.0    0.0   
16 00 000 001 000-PERSONA ATENDIDA PARA LA PREV...  99.0    0.0  80.0    0.0   
16 00 000 002 000-PERSONA CON DIAGNÓSTICO Y TRA...   1.0    0.0  20.0    0.0   
17 00 000 001 000-PERSONA INFORMADA SOBRE LA PR...   0.0    0.0   0.0  100.0   
17 00 000 002 000-PERSONA CON DIAGNÓSTICO DE LA...   0.0    0.0   0.0    0.0   

Year                                                        2017                
programa                                               18     16     17     18  
actividad_o                                                                     
18 00 000 001 000-PERSONA BENEFICIADA CON ACCIO...  100.0    0.0    0.0  100.0  
16 00 000 001 000-PERSONA ATENDIDA PARA LA PREV...    0.0  100.0    0.0    0.0  
16 00 000 002 000-PERSONA CON DIAGNÓSTICO Y TRA...    0.0    0.0    0.0    0.0  
17 00 000 001 000-PERSONA INFORMADA SOBRE LA PR...    0.0    0.0    0.0    0.0  
17 00 000 002 000-PERSONA CON DIAGNÓSTICO DE LA...    0.0    0.0  100.0    0.0

In [60]:
dataset.to_csv("../../Resource Tracking/prepped_data/SICOIN 2015-2017.csv")